In [3]:
from qtpy.QtGui import QFont, QImage, QPixmap, QColor
from qtpy.QtCore import Qt, Signal, QEvent, QTimer, QObject
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from qtpy.QtWidgets import (QPushButton, 
                            QComboBox, 
                            QSlider, 
                            QTextEdit, 
                            QPlainTextEdit, 
                            QWidget, 
                            QVBoxLayout, 
                            QHBoxLayout,
                            QLineEdit,
                            QLabel,
                            QFileDialog,     
                            QCheckBox,     
                            QDoubleSpinBox,
                            QSpinBox,    
                            QAbstractSpinBox,  
                            QMainWindow, 
                            QTableWidgetItem,
                            QTableWidget,    
                            QApplication, 
                            QMainWindow,    
                            QDialog,
                            QScrollArea, 
                            QSizePolicy,
                            QAbstractItemView                           
                            )
import numpy as np
import tifffile as tiff
import sys
import cv2
import porespy  as ps
import pandas as pd


c:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\venvs\sk_env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Volume filter

In [4]:
class ReadImage():
    def __init__(self, path):
        super().__init__()
        #Import image
        
        self.image_data = tiff.imread(path)

        # Normalize - generate dimension list (T,Z,H,W,C)
        self.dim = self.id_tiff_dim(path)
        # Reshape - STANDARDIZED 
        self.image_data = ((self.image_data / np.max(self.image_data)) * 255).astype(np.uint8)

        # IF shape: ZCXY ---------------------------------------------------------------
        # Change to TZXYC
        if self.image_data.shape[-3] <= 4:
            
            # dim = [1, 15, 1024, 1024, 3]
            print("Strange dimension")
            #if no time step data 
            if len(self.image_data.shape) == 4: 
                print(f'length {len(self.image_data.shape)}')
                self.image_data = self.image_data[np.newaxis,:,:,:,:]

            # Create an empty array with the specified dimensions
            image_data_stacked = np.empty(self.dim, dtype=np.uint8)

            num_time_frames = self.dim[0]  # Get the number of time frames
            
            for t in range(num_time_frames):  # Iterate over the time frames
                for i in range(self.dim[1]):  # Iterate over the images in each time frame
                    # Get the red, green, and blue channels for the i-th image in the t-th time frame
                    print(i)
                    chan_0 = self.image_data[t, i, 0, :, :]  # Assuming the first dimension is time frame, then image index
                    chan_1 = self.image_data[t, i, 1, :, :]
                    chan_2 = self.image_data[t, i, 2, :, :]
                    
                    # Stack the channels along the last axis
                    image_data_stacked[t, i, :, :, 0] = chan_0
                    image_data_stacked[t, i, :, :, 1] = chan_1
                    image_data_stacked[t, i, :, :, 2] = chan_2   
                    # if have a an alpha channel
                    if self.image_data.shape[2] >= 4:
                        chan_3 = self.image_data[t, i, 3, :, :] 
                        image_data_stacked[t, i, :, :, 3] = chan_3 

                    # if have a an alpha channel
                    if self.image_data.shape[2] == 5:
                        chan_4 = self.image_data[t, i, 4, :, :] 
                        image_data_stacked[t, i, :, :, 4] = chan_4 

            print(image_data_stacked.shape)   

            self.image_data = image_data_stacked 
            print(image_data_stacked.shape)
            self.slice = self.image_data[0,9,:,:,:]
            self.stack = self.image_data[0, :, :, :, :]
            print(f'shape: {self.slice.shape}, type: {type(self.slice)}')
        # ---------------------------------------------------------------------------------           
        
    def id_tiff_dim(self,f_path):
        tif_file = tiff.TiffFile(f_path)
        # Check for TIFF metadata tags
        metadata = tif_file.pages[0].tags
        if metadata:
            # print("Metadata Tags:")
            for tag_name, tag in metadata.items():
                print(f"{tag_name}: {tag.value}")

            #set dimension to 0 when a new tiff file is processed
            dimension = [1,1,1,1,1] #dim, slices , time
            
            
            #  T Z Y X C  (F, Z, H, W, C)
            #  0 1 2 3 4
            
            if 256 in metadata: #width
                            # Access the tag value directly
                            dimension[3] = metadata[256].value
            if 257 in metadata: #H
                            # Access the tag value directly
                            dimension[2] = metadata[257].value
            if 277 in metadata: #channels
                            # Access the tag value directly
                            dimension[4] = metadata[277].value
            if 259 in metadata:  # Tag for slices
                            print("meta",metadata[259].value)
                            dimension[1] = metadata[259].value
                        
            if 'ImageDescription' in metadata:
                    # Access 'ImageDescription' tag
                    image_description = metadata['ImageDescription']
            
                    # Split the 'ImageDescription' string into lines
                    description_lines = image_description.value.split('\n')
                    # Parse the lines to extract slices and frames information
                    for line in description_lines:
                        # if 262 in metadata:  # Tag for frames
                        #     dimension[4] = metadata[262].value
                        #     print("dim",dimension[4])
                        if line.startswith("slices="):
                            dimension[1] = int(line.split('=')[1]) #slice
                        if line.startswith("frames="):
                            dimension[0] = int(line.split('=')[1]) #frames
                            # print("frames", int(line.split('=')[1]))
                            # print("dim",dimension[4])
                        if line.startswith("channels="):
                            dimension[4] = int(line.split('=')[1]) #frames
                            # print("dim",dimension[4])
                        
        else:
                print("ImageDescription tag not found in metadata.")
                        
        # print(f'Width: {dimension[3]}')
        # print(f'Height: {dimension[2]}')
        # print(f'Channels: {dimension[4]}')
        # print(f"Slices: {dimension[1]}")
        # print(f"Frames: {dimension[0]}")
        # print(f'Dimension: {dimension[0]}')
        # self.dimension=dimension
        # set_widg = [1,1]
        # set_widg[0] = dimension[0] #t
        # set_widg[1] = dimension[1] #z
        # self.SIGNALS.reset_widget.emit(set_widg)
        # self.SIGNALS.image_shape.emit(dimension)
        # self.stack_dict["time_step"]= 1
        # self.zzval= round(set_widg[1]/2)
        # if (dimension[0])==1 or (dimension[1])==1:
        #     set_widg = [1,1]
        #     self.SIGNALS.reset_widget.emit(set_widg)
        #     self.stack_dict["time_step"]=0
        #     self.zzval=0
        print(f'Image dim: {dimension}')
        return dimension

image_filepath = r"C:\Users\dell\Downloads\Rapa+Baf_3\Rapa+Baf_3.tif"
# image_filepath = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"
# image_filepath = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\venvs\sk_env2\Lib\site-packages\ryven\skripsie_projects\batch_con\Con 1 (20.3).tif"
read_img = ReadImage(image_filepath)
# extract self.slice (object)
# single slice (chose 1, 9, : , :, :)
# slicexyx = read_img.slice
# print(slicexyx.shape)


254: 0
256: 1024
257: 1024
258: 8
262: 1
270: ImageJ=1.53j
images=120
channels=4
slices=30
hyperstack=true
mode=composite
loop=false
273: (3426,)
277: 1
278: 1024
279: (1048576,)
50838: (20, 64, 768, 768, 768, 768)
50839: {'Ranges': (0.0, 255.0, 0.0, 255.0, 0.0, 255.0, 0.0, 255.0), 'LUTs': [array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125

In [11]:
control_2_stack = read_img.stack
print("control_2_stack", control_2_stack.shape)
chan_1_stack = control_2_stack[:,:,:,1]
print("channel 1 stack:", chan_1_stack.shape)
chan_1_img_stack = chan_1_stack[:, :, :, np.newaxis]
print("channel 1 stack:", chan_1_img_stack.shape)

control_2_stack (15, 1024, 1024, 3)
channel 1 stack: (15, 1024, 1024)
channel 1 stack: (15, 1024, 1024, 1)


In [12]:
binaized_stack = []
for i, stack in enumerate(chan_1_img_stack):
    print(i, stack.shape)

0 (1024, 1024, 1)
1 (1024, 1024, 1)
2 (1024, 1024, 1)
3 (1024, 1024, 1)
4 (1024, 1024, 1)
5 (1024, 1024, 1)
6 (1024, 1024, 1)
7 (1024, 1024, 1)
8 (1024, 1024, 1)
9 (1024, 1024, 1)
10 (1024, 1024, 1)
11 (1024, 1024, 1)
12 (1024, 1024, 1)
13 (1024, 1024, 1)
14 (1024, 1024, 1)


In [13]:
img_gray = cv2.cvtColor(control_2_stack[0, :,:,:], cv2.COLOR_BGR2GRAY)
            #print(f"SHAPE OF PROC {img.shape}")    
_, result = cv2.threshold(
    src=img_gray,
    thresh=100,
    maxval=255,
    type=cv2.THRESH_BINARY,
)

In [14]:
import numpy as np
from scipy.ndimage import gaussian_filter, label, sum
# img = read_img[0, 9, :, :, :]
# print(img.shape)
# Binarize image

binarized_stack = np.zeros((15, 1024, 1024), dtype=np.uint8)

for i in range(15):
    img = control_2_stack[i,:,:,:]

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, result = cv2.threshold(
        src=img_gray,
        thresh=100,
        maxval=255,
        type=cv2.THRESH_BINARY,
    )
    
    
    binarized_stack[i] = result

print(binarized_stack.shape)
binarisedImageStack = binarized_stack[:, :,:, np.newaxis]
print(binarisedImageStack.shape)

(15, 1024, 1024)
(15, 1024, 1024, 1)


In [15]:
import numpy as np
from scipy.ndimage import gaussian_filter, label, sum
# img = read_img[0, 9, :, :, :]
# print(img.shape)
# Binarize image
# binaized_stack = []
# for i, stack in enumerate(chan_1_stack):
#     img_gray = cv2.cvtColor(stack, cv2.COLOR_BGR2GRAY)
#             #print(f"SHAPE OF PROC {img.shape}")    
#     _, result = cv2.threshold(
#         src=img_gray,
#         thresh=100,
#         maxval=255,
#         type=cv2.THRESH_BINARY,
#     )
#     binaized_stack[i]=result
    # binarisedImageStack = result[:, :, np.newaxis]

# Assuming self.image_stack contains the binarized image stack
# squeeze = np.squeeze(img_rehsape)
# squeeze = squeeze.astype(int)  # Convert to int if not already

# # Smooth the image using Gaussian filter (optional)
# smoothed_image = gaussian_filter(squeeze, sigma=1)

## Lable node ------------------------------------

# Label connected components
labeled, numpatches = label(binarisedImageStack)

# Select min volume
minVolume = 40

# since labels start from 1 use this range
sizes = sum(binarisedImageStack/np.max(binarisedImageStack), labeled, range(1, numpatches + 1))
# print(np.sort(sizes.astype('uint32')))

# to ensure "black background" is excluded add 1, and labels only start from 1
filteredIndexes = np.where(sizes >= minVolume)[0] + 1

filteredBinaryIndexes = np.zeros(numpatches + 1, np.uint8)
filteredBinaryIndexes[filteredIndexes] = 1
filteredBinary = filteredBinaryIndexes[labeled]

labeledStack, numLabels = label(filteredBinary)
print("Initial num labels: {}, num lables after filter: {}".format(numpatches, numLabels))

# sizes = ndimage.sum(filteredBinary/np.max(filteredBinary), labeledStack, range(1, numLabels + 1))
# print(np.sort(sizes.astype('uint32')))

# return filteredBinary, labeledStack, numLabels


# ------------------------------------------------

# # Selected value for structure size
# selected_value = 1  # Number of pixels

# # Initialize the count of structures
# num_structures = 0

# # Loop through each labeled region
# # num_features represents the total number of unique labels in the image.
# for label_id in range(1, num_features+1):
#     # Calculate the area (number of pixels) of the current labeled region
#     print(label_id, labeled)
#     region_area = np.sum(labeled == label_id) # This comparison creates a boolean mask where True corresponds 
#                                               # to pixels belonging to the current labeled region (with label ID label_id) 
#                                               # and False otherwise. Sum these to find the size of the area. 

    
#     # Check if the region's area is greater than the selected value
#     if region_area >= selected_value:
#         # Increment the count of structures
#         num_structures += 1

# print("Number of structures:", num_structures)

Initial num labels: 464, num lables after filter: 64


In [ ]:
binarisedImageStack.shape

(15, 1024, 1024, 1)

In [ ]:
type(binarisedImageStack)

numpy.ndarray

## Metadata Properties

In [ ]:
class OutputMetadata():
    def __init__(self, img):
        super().__init__()

        self.image_stack = img
        self.properties()

    def properties(self):
            squeeze = np.squeeze(self.image_stack)
            squeeze = squeeze.astype(int)  #solved -- NB
            # print(squeeze.shape)
            # print(squeeze.dtype)
            labeled, numfeatures = label(squeeze)
            properties = ps.metrics.regionprops_3D(labeled)

            # Create an empty list to store dictionaries of property values
            property_dicts = []

            #loop through regions
            for pr in properties:
                property_dict = {
                    'Label': pr.label,
                    'Area': pr.area,
                    'Centroid': pr.centroid,
                    'Equivalent Diameter': pr.equivalent_diameter,
                    'Euler Number': pr.euler_number,
                    'Extent': pr.extent,
                    'Filled Area': pr.filled_area,
                    'Inertia Tensor Eigvals': pr.inertia_tensor_eigvals,
                    'Volume (Physical Space)': pr.volume,
                    'Surface Area (Physical Space)': pr.surface_area,
                    'Sphericity (Physical Space)': pr.sphericity,
                    'Aspect Ratio (Major/Minor)': np.sqrt(pr.inertia_tensor_eigvals[0] / pr.inertia_tensor_eigvals[-1])
                }
                property_dicts.append(property_dict)

                # print('append')

            # Create a pandas DataFrame from the list of property dictionaries
            df = pd.DataFrame(property_dicts)
            summary_metadata = {}
            # Calculate the statistics
            total_structures = numfeatures  # Assuming 'df' is your existing DataFrame #rows
            area_avg = df["Area"].mean()
            centroid_avg = np.mean(df['Centroid'].to_list(), axis=0)
            equivalent_diameter_avg = df["Equivalent Diameter"].mean()
            euler_avg = df["Euler Number"].mean()
            extent_avg = df["Extent"].mean()
            filled_area_avg = df["Filled Area"].mean()
            inertia_avg = np.mean(df['Inertia Tensor Eigvals'].to_list(), axis=0)
            volume_avg = df['Volume (Physical Space)'].mean()
            surface_area_avg = df['Surface Area (Physical Space)'].mean()
            sphericity_avg = df['Sphericity (Physical Space)'].mean()
            aspect_ratio_avg = df['Aspect Ratio (Major/Minor)'].mean()

                       
            summary_metadata = {
                'Timestep': 0,
                'Total Structures': total_structures,
                'Area Avg': area_avg,
                'Centroid Avg': tuple(centroid_avg),
                'Equivalent Diameter Avg': equivalent_diameter_avg,
                'Euler Number Avg': euler_avg,
                'Extent Avg': extent_avg,
                'Filled Area Avg': filled_area_avg,
                'Inertia Tensor Eigenvalues Avg': tuple(inertia_avg),
                'Volume Avg (Physical Space)': volume_avg,
                'Surface Area Avg (Physical Space)': surface_area_avg,
                'Sphericity Avg (Physical Space)': sphericity_avg,
                'Aspect Ratio Avg (Major/Minor)': aspect_ratio_avg
            }

            def convert_to_string(coloumn):
                return coloumn.apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            # Formatting the 'Centroid' and 'Inertia Tensor Eigvals' columns
            # Coverts tuple lists to string
            df['Centroid'] = df['Centroid'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            df['Inertia Tensor Eigvals'] = df['Inertia Tensor Eigvals'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            # print(df)
            print(summary_metadata)
            self.summary_metadata_df = pd.DataFrame([summary_metadata])
            self.summary_metadata_df['Centroid Avg'] = self.summary_metadata_df['Centroid Avg'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            self.summary_metadata_df['Inertia Tensor Eigenvalues Avg'] = self.summary_metadata_df['Inertia Tensor Eigenvalues Avg'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            # df['Centroid'] = df['Centroid'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            # df['Inertia Tensor Eigvals'] = df['Inertia Tensor Eigvals'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
        

my_props = OutputMetadata(binarisedImageStack)

df = my_props.summary_metadata_df
df

{'Timestep': 0, 'Total Structures': 464, 'Area Avg': 17.834051724137932, 'Centroid Avg': (5.073049734062022, 483.2219390034362, 419.8420927874811), 'Equivalent Diameter Avg': 2.4150695340725914, 'Euler Number Avg': 0.9612068965517241, 'Extent Avg': 0.7652916211166431, 'Filled Area Avg': 17.834051724137932, 'Inertia Tensor Eigenvalues Avg': (1.754462970240409, 1.2999550309352839, 0.6365924067206375), 'Volume Avg (Physical Space)': 17.834051724137932, 'Surface Area Avg (Physical Space)': 37.992821316266884, 'Sphericity Avg (Physical Space)': 0.8670581316534879, 'Aspect Ratio Avg (Major/Minor)': inf}


,Timestep,Total Structures,Area Avg,Centroid Avg,Equivalent Diameter Avg,Euler Number Avg,Extent Avg,Filled Area Avg,Inertia Tensor Eigenvalues Avg,Volume Avg (Physical Space),Surface Area Avg (Physical Space),Sphericity Avg (Physical Space),Aspect Ratio Avg (Major/Minor)
0,0,464,17.834052,"5.073049734062022, 483.2219390034362, 419.8420...",2.41507,0.961207,0.765292,17.834052,"1.754462970240409, 1.2999550309352839, 0.63659...",17.834052,37.992821,0.867058,inf


In [ ]:
import os 
filename_morph = "test.tiff"
morph_output_path = "downloads\here"
base_filename = os.path.splitext(filename_morph)[0]
csv_filename = f"{base_filename}.csv"
new_morph_path = os.path.join(morph_output_path, csv_filename)
# df.to_csv(self.new_morph_path, index =False)
new_morph_path

'downloads\\here\\test.csv'

In [ ]:

class OutputMetadata():
    def __init__(self, img):
        super().__init__()

        self.image_stack = img
        self.properties_summary()

    def properties_summary(self):
            squeeze = np.squeeze(self.image_stack)
            squeeze = squeeze.astype(int)  #solved -- NB
            # print(squeeze.shape)
            # print(squeeze.dtype)
            labeled, numfeatures = label(squeeze)
            properties = ps.metrics.regionprops_3D(labeled)

            # Create an empty list to store dictionaries of property values
            property_dicts = []

            #loop through regions
            for pr in properties:
                property_dict = {
                    'Label': pr.label,
                    'Area': pr.area,
                    'Centroid': pr.centroid,
                    'Equivalent Diameter': pr.equivalent_diameter,
                    'Euler Number': pr.euler_number,
                    'Extent': pr.extent,
                    'Filled Area': pr.filled_area,
                    'Inertia Tensor Eigvals': pr.inertia_tensor_eigvals,
                    'Volume (Physical Space)': pr.volume,
                    'Surface Area (Physical Space)': pr.surface_area,
                    'Sphericity (Physical Space)': pr.sphericity,
                    'Aspect Ratio (Major/Minor)': np.sqrt(pr.inertia_tensor_eigvals[0] / pr.inertia_tensor_eigvals[-1])
                }
                property_dicts.append(property_dict)

            # Create a pandas DataFrame from the list of property dictionaries
            df = pd.DataFrame(property_dicts)
            
            # Calculate the statistics
            total_structures = numfeatures  # Assuming 'df' is your existing DataFrame #rows
            area_avg = df["Area"].mean()
            centroid_avg = np.mean(df['Centroid'].to_list(), axis=0)
            equivalent_diameter_avg = df["Equivalent Diameter"].mean()
            euler_avg = df["Euler Number"].mean()
            extent_avg = df["Extent"].mean()
            filled_area_avg = df["Filled Area"].mean()
            inertia_avg = np.mean(df['Inertia Tensor Eigvals'].to_list(), axis=0)
            volume_avg = df['Volume (Physical Space)'].mean()
            surface_area_avg = df['Surface Area (Physical Space)'].mean()
            sphericity_avg = df['Sphericity (Physical Space)'].mean()
            aspect_ratio_avg = df['Aspect Ratio (Major/Minor)'].mean()
            

            summary_metadata = []
            summary_metadata_dict = {
                'Timestep': 0, # self.stack_dict["time_step"]
                'Total Structures': total_structures,
                'Area Avg': area_avg,
                'Centroid Avg': tuple(centroid_avg),
                'Equivalent Diameter Avg': equivalent_diameter_avg,
                'Euler Number Avg': euler_avg,
                'Extent Avg': extent_avg,
                'Filled Area Avg': filled_area_avg,
                'Inertia Tensor Eigenvalues Avg': tuple(inertia_avg),
                'Volume Avg (Physical Space)': volume_avg,
                'Surface Area Avg (Physical Space)': surface_area_avg,
                'Sphericity Avg (Physical Space)': sphericity_avg,
                'Aspect Ratio Avg (Major/Minor)': aspect_ratio_avg
            }
            summary_metadata.append(summary_metadata_dict)
            
            
            print(summary_metadata)
            self.summary_metadata_df = pd.DataFrame(summary_metadata)
            self.summary_metadata_df['Centroid Avg'] = self.summary_metadata_df['Centroid Avg'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            self.summary_metadata_df['Inertia Tensor Eigenvalues Avg'] = self.summary_metadata_df['Inertia Tensor Eigenvalues Avg'].apply(lambda x: f"{x[0]}, {x[1]}, {x[2]}")
            
            # path = r"c:\Users\dell\Downloads"
            # summary_path = path + "/time_series_summary.csv"
       
            # summary_csv = self.summary_metadata_df.to_csv(summary_path, index =False)
            path_x = r"c:\Users\dell\Downloads\summary.xlsx"
            summary_csv = self.summary_metadata_df.to_excel(path_x, index =False)

my_props = OutputMetadata(binarisedImageStack)
my_props.summary_metadata_df

[{'Timestep': 0, 'Total Structures': 464, 'Area Avg': 17.834051724137932, 'Centroid Avg': (5.073049734062022, 483.2219390034362, 419.8420927874811), 'Equivalent Diameter Avg': 2.4150695340725914, 'Euler Number Avg': 0.9612068965517241, 'Extent Avg': 0.7652916211166431, 'Filled Area Avg': 17.834051724137932, 'Inertia Tensor Eigenvalues Avg': (1.754462970240409, 1.2999550309352839, 0.6365924067206375), 'Volume Avg (Physical Space)': 17.834051724137932, 'Surface Area Avg (Physical Space)': 37.992821316266884, 'Sphericity Avg (Physical Space)': 0.8670581316534879, 'Aspect Ratio Avg (Major/Minor)': inf}]


,Timestep,Total Structures,Area Avg,Centroid Avg,Equivalent Diameter Avg,Euler Number Avg,Extent Avg,Filled Area Avg,Inertia Tensor Eigenvalues Avg,Volume Avg (Physical Space),Surface Area Avg (Physical Space),Sphericity Avg (Physical Space),Aspect Ratio Avg (Major/Minor)
0,0,464,17.834052,"5.073049734062022, 483.2219390034362, 419.8420...",2.41507,0.961207,0.765292,17.834052,"1.754462970240409, 1.2999550309352839, 0.63659...",17.834052,37.992821,0.867058,inf


In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, label
# img = read_img[0, :, :, :, :]
# print(img.shape)
img_gray = cv2.cvtColor(slicexyx, cv2.COLOR_BGR2GRAY)
        #print(f"SHAPE OF PROC {img.shape}")    
_, result = cv2.threshold(
    src=img_gray,
    thresh=100,
    maxval=255,
    type=cv2.THRESH_BINARY,
)
img_rehsape = result[:, :, np.newaxis]

# Assuming self.image_stack contains the binarized image stack
squeeze = np.squeeze(img_rehsape)
squeeze = squeeze.astype(int)  # Convert to int if not already

# # Smooth the image using Gaussian filter (optional)
# smoothed_image = gaussian_filter(squeeze, sigma=1)

## Lable node ------------------------------------
# Label connected components
labeled, num_features = label(squeeze)

# Calculate region areas
region_areas = np.bincount(labeled.ravel())[1:]


# Create a mask to filter regions below the volume threshold
mask = np.zeros_like(labeled, dtype=bool)
mask[np.isin(labeled, np.nonzero(region_areas >= 5)[0] + 1)] = True

# Apply the mask to the original image
filtered_image = squeeze * mask.astype(squeeze.dtype)

# Count the number of structures
num_structures = np.count_nonzero(mask)

# Print the number of structures
print("Number of structures:", num_structures)

In [ ]:
def id_tiff_dim(f_path):
        tif_file = tiff.TiffFile(f_path)
        # Check for TIFF metadata tags
        metadata = tif_file.pages[0].tags
        if metadata:
            # print("Metadata Tags:")
            for tag_name, tag in metadata.items():
                print(f"{tag_name}: {tag.value}")

            #set dimension to 0 when a new tiff file is processed
            dimension = [1,1,1,1,1] #dim, slices , time
            
            
            #  T Z Y X C  (F, Z, H, W, C)
            #  0 1 2 3 4
            
            if 256 in metadata: #width
                            # Access the tag value directly
                            dimension[3] = metadata[256].value
            if 257 in metadata: #H
                            # Access the tag value directly
                            dimension[2] = metadata[257].value
            if 277 in metadata: #channels
                            # Access the tag value directly
                            dimension[4] = metadata[277].value
            if 259 in metadata:  # Tag for slices
                            print("meta",metadata[259].value)
                            dimension[1] = metadata[259].value
                        
            if 'ImageDescription' in metadata:
                    # Access 'ImageDescription' tag
                    image_description = metadata['ImageDescription']
            
                    # Split the 'ImageDescription' string into lines
                    description_lines = image_description.value.split('\n')
                    # Parse the lines to extract slices and frames information
                    for line in description_lines:
                        # if 262 in metadata:  # Tag for frames
                        #     dimension[4] = metadata[262].value
                        #     print("dim",dimension[4])
                        if line.startswith("slices="):
                            dimension[1] = int(line.split('=')[1]) #slice
                        if line.startswith("frames="):
                            dimension[0] = int(line.split('=')[1]) #frames
                            # print("frames", int(line.split('=')[1]))
                            # print("dim",dimension[4])
                        if line.startswith("channels="):
                            dimension[4] = int(line.split('=')[1]) #frames
                            # print("dim",dimension[4])
                        
        else:
                print("ImageDescription tag not found in metadata.")
                        
       
        set_widg = [1,1]
        set_widg[0] = dimension[0] #t
        set_widg[1] = dimension[1] #z
      
        return dimension

In [ ]:
#input image
image_filepath = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\venvs\sk_env2\Lib\site-packages\ryven\skripsie_projects\Control_2.tif"
image_data = tiff.imread(image_filepath)
image_data = ((image_data / np.max(image_data)) * 255).astype(np.uint8)
print(image_data.shape)
print(len(image_data[2]))

#find dimentions 
dim = id_tiff_dim(image_filepath)
print(dim)

if image_data.shape[-3] <= 4:
                    
                    # dim = [1, 15, 1024, 1024, 3]
                    print("Strange dimension")
                    #if no time step data 
                    if len(image_data.shape) == 4: 
                        print(f'length {len(image_data.shape)}')
                        image_data = image_data[np.newaxis,:,:,:,:]

                    # Create an empty array with the specified dimensions
                    image_data_stacked = np.empty(dim, dtype=np.uint8)

                    num_time_frames = dim[0]  # Get the number of time frames
                    
                    for t in range(num_time_frames):  # Iterate over the time frames
                        for i in range(dim[1]):  # Iterate over the images in each time frame
                            # Get the red, green, and blue channels for the i-th image in the t-th time frame
                            print(i)
                            chan_0 = image_data[t, i, 0, :, :]  # Assuming the first dimension is time frame, then image index
                            chan_1 = image_data[t, i, 1, :, :]
                            chan_2 = image_data[t, i, 2, :, :]
                            
                            # Stack the channels along the last axis
                            image_data_stacked[t, i, :, :, 0] = chan_0
                            image_data_stacked[t, i, :, :, 1] = chan_1
                            image_data_stacked[t, i, :, :, 2] = chan_2   
                            # if have a an alpha channel
                            if image_data.shape[2] >= 4:
                                chan_3 = image_data[t, i, 3, :, :] 
                                image_data_stacked[t, i, :, :, 3] = chan_3 

                            # if have a an alpha channel
                            if image_data.shape[2] == 5:
                                chan_4 = image_data[t, i, 4, :, :] 
                                image_data_stacked[t, i, :, :, 4] = chan_4 

                    print(image_data_stacked.shape)   

                    image_data = image_data_stacked 
                # ---------------------------------------------------------------------------------           
                    
reshaped_data = image_data.reshape(dim)
print("reshaped", reshaped_data.shape)

In [ ]:


# path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\venvs\sk_env2\Lib\site-packages\ryven\skripsie_projects\Control_2.tif"
# tif_file = tiff.TiffFile(path)
# image = tiff.imread(path)

# # metadata = tif_file.pages[0].tags
# # if metadata:
# #      # print("Metadata Tags:")
# #     for tag_name, tag in metadata.items():
# #         print(f"{tag_name}: {tag.value}")

# print(image.shape)
# print(np.max(image[9,0,:,:]))
# print(np.max(image[9,1,:,:]))
# print(np.max(image[9,2,:,:]))
# # Slice
# img = image[:, :, :, :]
# # Normlize for 8bit PyQt widget
# img_norm = ((img / np.max(img)) * 255).astype(np.uint8)
# print("Img norm", img_norm.shape)
img = reshaped_data[0, :, :, :, :]
print(img.shape)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #print(f"SHAPE OF PROC {img.shape}")    
_, result = cv2.threshold(
    src=img_gray,
    thresh=100,
    maxval=255,
    type=cv2.THRESH_BINARY,
)
img_rehsape = result[:, :, np.newaxis]

# Assuming self.image_stack contains the binarized image stack
squeeze = np.squeeze(img_rehsape)
squeeze = squeeze.astype(int)  # Convert to int if not already

# Smooth the image using Gaussian filter (optional)
smoothed_image = gaussian_filter(squeeze, sigma=1)

# Label connected components
labeled, num_features = label(smoothed_image)

# Selected value for structure size
selected_value = 100  # Number of pixels

# Initialize the count of structures
num_structures = 0

# Loop through each labeled region
for label_id in range(1, num_features+1):
    # Calculate the area (number of pixels) of the current labeled region
    region_area = np.sum(labeled == label_id)
    
    # Check if the region's area is greater than the selected value
    if region_area >= selected_value:
        # Increment the count of structures
        num_structures += 1

print("Number of structures:", num_structures)

### Export to CSV with different sheets

In [ ]:

data = {
    'Name': ['John', 'Alice', 'Bob'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
}



data2 = {
    'Name': ['Emma', 'Olivia', 'Dave'],
    'Age': [1, 2, 3],
    'City': ['Stb', 'Los Angeles', 'CT']
}

df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
df2

In [ ]:
output_path = r"C:\Users\dell\Downloads"
csv_file1 = output_path + r"\csvd1.csv"
df1.to_csv(csv_file1, index=False)


In [ ]:
import win32com.client

# Path to the CSV file
csv_file_path = r'C:\Users\dell\Downloads\csvd1.csv'

# Start Excel application
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False  # Make Excel visible (optional)

# Open the CSV file in Excel
workbook = excel.Workbooks.OpenText(
    csv_file_path,
    DataType=1,  # Delimited
    TextQualifier=1,  # Double quote
    ConsecutiveDelimiter=True,
    Tab=False,  # Not using tab delimiter
    Semicolon=False,  # Not using semicolon delimiter
    Comma=True  # Comma delimiter
)

# Save the workbook (optional)
workbook.SaveAs(r'C:\Users\dell\Downloads\excel_file.xlsx')

# Close the workbook and Excel application
workbook.Close()
excel.Quit()


In [ ]:
import win32com.client

# Path to the CSV file
csv_file_path = r'C:\Users\dell\Downloads\csvd1.csv'
# Start Excel application
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True  # Make Excel visible (optional)

# Open the CSV file in Excel
workbook = excel.Workbooks.Open(csv_file_path)

# Save the workbook as an Excel file
excel_file_path = r'C:\Users\dell\Downloads\excel_file.xlsx'
workbook.SaveAs(excel_file_path, FileFormat=51)  # FileFormat=51 for .xlsx format

# Close the workbook and Excel application
workbook.Close()
excel.Quit()

print(f"CSV file '{csv_file_path}' successfully saved as Excel file '{excel_file_path}'.")


In [ ]:


# Sample DataFrames
data = {
    'Sheet1': pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}),
    'Sheet2': pd.DataFrame({'C': [7, 8, 9], 'D': [10, 11, 12]})
}

# Write each DataFrame to a separate CSV file
for sheet_name, df in data.items():
    # Define the file path for each CSV file
    csv_file = f'{sheet_name}.csv'
    # Write the DataFrame to the CSV file
    df.to_csv(csv_file, index=False)

print("CSV files created successfully.")


In [ ]:
def check(row_m,col_m, row, col):
    max = row_m+col_m
    print(max)

In [ ]:
check(1,10,2,3)

In [ ]:
# self.stack_dict = {
#             "time_step": self.frame,
#             "colour": {
#                 "red": 100, #If that channel does not exsit set it to 100 
#                 "green": 100,
#                 "blue": 100,
#                 "magenta": 100,
#                 "cyan": 100
#             }
#         }

### Image colour prototype

Import image and prepare

In [ ]:
def id_tiff_dim(f_path):
        tif_file = tiff.TiffFile(f_path)
        # Check for TIFF metadata tags
        metadata = tif_file.pages[0].tags
        if metadata:
            # print("Metadata Tags:")
            for tag_name, tag in metadata.items():
                print(f"{tag_name}: {tag.value}")

            #set dimension to 0 when a new tiff file is processed
            dimension = [1,1,1,1,1] #dim, slices , time
            
            
            #  T Z Y X C  (F, Z, H, W, C)
            #  0 1 2 3 4
            
            if 256 in metadata: #width
                            # Access the tag value directly
                            dimension[3] = metadata[256].value
            if 257 in metadata: #H
                            # Access the tag value directly
                            dimension[2] = metadata[257].value
            if 277 in metadata: #channels
                            # Access the tag value directly
                            dimension[4] = metadata[277].value
            if 259 in metadata:  # Tag for slices
                            print("meta",metadata[259].value)
                            dimension[1] = metadata[259].value
                        
            if 'ImageDescription' in metadata:
                    # Access 'ImageDescription' tag
                    image_description = metadata['ImageDescription']
            
                    # Split the 'ImageDescription' string into lines
                    description_lines = image_description.value.split('\n')
                    # Parse the lines to extract slices and frames information
                    for line in description_lines:
                        # if 262 in metadata:  # Tag for frames
                        #     dimension[4] = metadata[262].value
                        #     print("dim",dimension[4])
                        if line.startswith("slices="):
                            dimension[1] = int(line.split('=')[1]) #slice
                        if line.startswith("frames="):
                            dimension[0] = int(line.split('=')[1]) #frames
                            # print("frames", int(line.split('=')[1]))
                            # print("dim",dimension[4])
                        if line.startswith("channels="):
                            dimension[4] = int(line.split('=')[1]) #frames
                            # print("dim",dimension[4])
                        
        else:
                print("ImageDescription tag not found in metadata.")
                        
       
        set_widg = [1,1]
        set_widg[0] = dimension[0] #t
        set_widg[1] = dimension[1] #z
      
        return dimension

In [ ]:
#input image
image_filepath = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\venvs\sk_env2\Lib\site-packages\ryven\skripsie_projects\Control_2.tif"
image_data = tiff.imread(image_filepath)
image_data = ((image_data / np.max(image_data)) * 255).astype(np.uint8)
print(image_data.shape)
print(len(image_data[2]))

#find dimentions 
dim = id_tiff_dim(image_filepath)
print(dim)

#select slice
img = image_data[9,:,:,:]

# IF shape: ZCXY ---------------------------------------------------------------
# Change to TZXYC
if image_data.shape[-3] <= 4:
    
    # dim = [1, 15, 1024, 1024, 3]
    print("Strange dimension")
    #if no time step data 
    if len(image_data.shape) == 4: 
        print(f'length {len(image_data.shape)}')
        image_data = image_data[np.newaxis,:,:,:,:]

    # Create an empty array with the specified dimensions
    image_data_stacked = np.empty(dim, dtype=np.uint8)

    num_time_frames = dim[0]  # Get the number of time frames
    
    for t in range(num_time_frames):  # Iterate over the time frames
        for i in range(dim[1]):  # Iterate over the images in each time frame
            # Get the red, green, and blue channels for the i-th image in the t-th time frame
            print(i)
            chan_0 = image_data[t, i, 0, :, :]  # Assuming the first dimension is time frame, then image index
            chan_1 = image_data[t, i, 1, :, :]
            chan_2 = image_data[t, i, 2, :, :]
            
            # Stack the channels along the last axis
            image_data_stacked[t, i, :, :, 0] = chan_0
            image_data_stacked[t, i, :, :, 1] = chan_1
            image_data_stacked[t, i, :, :, 2] = chan_2   
            # if have a an alpha channel
            if image_data.shape[2] >= 4:
                chan_3 = image_data[t, i, 3, :, :] 
                image_data_stacked[t, i, :, :, 3] = chan_3 

            # if have a an alpha channel
            if image_data.shape[2] == 5:
                chan_4 = image_data[t, i, 4, :, :] 
                image_data_stacked[t, i, :, :, 4] = chan_4 

print(image_data_stacked.shape)


Select image slice 

In [ ]:
img = image_data_stacked[0,9,:,:,:]

Widget

In [ ]:
class Widget(QWidget):
    def __init__(self):
        super().__init__()

        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100, #If that channel does not exsit set it to 100 
                "green": 1,
                "blue": 2,
                "magenta": 0,
                "cyan": 100
            }
        }
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):

        # Extract individual channels
        magenta_channel = img[:,:,0]
        green_channel = img[:,:,1]
        blue_channel = img[:,:,2]
        red_channel = np.zeros_like(blue_channel)

        # Add magenta channel to red and blue channels
        red_channel = np.add(red_channel, magenta_channel)
        blue_channel = np.add(blue_channel, magenta_channel)

        # Clip values to ensure they remain within the valid range [0, 255]
        red_channel = np.clip(red_channel, 0, 255)
        blue_channel = np.clip(blue_channel, 0, 255)

        # Combine channels back into image data
        img = np.stack([red_channel, green_channel, blue_channel], axis=2)
        # try:
        if img.shape[-1] == 1:
            # Grayscale image
            qt_image = QImage(img.data, img.shape[1], img.shape[0], img.shape[1], QImage.Format_Grayscale8)
            # #print("came here for Sliderwidget")
        elif img.shape[-1] == 3:
            h, w, ch = img.shape
            bytes_per_line = ch * w
            qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888) #Format_RGB888
        elif img.shape[-1] == 4:
            h, w, ch = img.shape
            #print(f"ch: {ch}")
            bytes_per_line = ch * 4
            qt_image = QImage(img.data, w, h, QImage.Format_RGBA8888) #Format_RGB888
        if qt_image is not None:
            # Calculate the target size for scaling
            scale_factor = 0.7  # Increase the scaling factor for clarity
            if qt_image.width() < 400:
                scale_factor = 1
            if qt_image.width() > 900:
                scale_factor = 0.5
            target_width = int(qt_image.width() * scale_factor)
            # Use scaledToWidth to reduce the size while maintaining aspect ratio
            scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
            
            # Set the scaled pixmap
            self.image_label.setPixmap(scaled_pixmap)
          

# Create an instance of the QApplication object
app = QApplication(sys.argv)

# Create an instance of the widget
widget = Widget()

# Show the image in the widget
widget.show_image(img)

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())

In [ ]:


image_label = QLabel()
layout1 = QVBoxLayout()
layout1.addWidget(image_label)  

if img.shape[-1] == 1:
    # Grayscale image
    qt_image = QImage(img.data, img.shape[1], img.shape[0], img.shape[1], QImage.Format_Grayscale8)
    # #print("came here for Sliderwidget")
elif img.shape[-1] == 3:
    h, w, ch = img.shape
    bytes_per_line = ch * w
    qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888) #Format_RGB888
elif img.shape[-1] == 4:
    h, w, ch = img.shape
    #print(f"ch: {ch}")
    bytes_per_line = ch * 4
    qt_image = QImage(img.data, w, h, QImage.Format_RGBA8888) #Format_RGB888
if qt_image is not None:
    # Calculate the target size for scaling
    scale_factor = 0.7  # Increase the scaling factor for clarity
    if qt_image.width() < 400:
        scale_factor = 1
    if qt_image.width() > 900:
        scale_factor = 0.5
    target_width = int(qt_image.width() * scale_factor)
    # Use scaledToWidth to reduce the size while maintaining aspect ratio
    scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
    
    # Set the scaled pixmap
    image_label.setPixmap(scaled_pixmap)
    
    # # Resize the widget to match the pixmap size
    # self.resize(scaled_pixmap.width(), scaled_pixmap.height())
    
    # # Ensure that any necessary updates are performed
    # self.node.update_shape()

# except Exception as e:
#         print("Error:", e)

### Batch Processing

Select the midpoint indices for the slice to be displayed in the pipeline

In [ ]:
dimension = [1,15,512,512,3]
zzval = int(dimension[1] /2)
stack = int(dimension[0] / 2)
print(zzval, stack)

In [ ]:
import os

folder_path = r'C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf'

# Loop through files in the folder
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        print(filename)
        file = os.path.join(folder_path, filename)
        print(file)
        filename = filename

new_folder_name = "BatchProcessed"  # You can change this to your desired folder name
new_folder_path = os.path.join(folder_path, new_folder_name)
print(new_folder_path)
# os.makedirs(new_folder_path)

new_file_path = os.path.join(new_folder_path, "batched_processed_" + filename)
print(new_file_path)
i = 0

Baf 1 (20.3).tif
C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf\Baf 1 (20.3).tif
Baf 2 (20.3).tif
C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf\Baf 2 (20.3).tif
Baf3 (20.3).tif
C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf\Baf3 (20.3).tif
C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf\BatchProcessed
C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Deconvolved\Baf\BatchProcessed\batched_processed_Baf3 (20.3).tif


'C:\\Users\\dell\\OneDrive\\Documents\\Em\\2023\\Skripsie\\Development\\Data\\Deconvolved\\Baf\\BatchProcessed\\batched_processed_Baf3 (20.3).tif'

In [ ]:
tuples = (1,3)
tuples[0]

In [ ]:
files = os.listdir(folder_path)
tiff_files = [file for file in files if file.lower().endswith(('.tif', '.tiff'))]
num_tiff = len(tiff_files)
print(i)
if i < num_tiff:
        print('perform')
        firstfile = tiff_files[i]
        print(firstfile)
        i+=1
# else:
#         return
print(i)

Save RGB images

In [ ]:
import tifffile as tiff
import numpy as np

path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\fly\flybrain.tif"
tif_file = tiff.TiffFile(path)
img = tiff.imread(path)
img_norm = ((img / np.max(img)) * 255).astype(np.uint8)
print(img_norm.shape)
new_file_path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\fly\BatchProcessed\flybrainY.tif"
custom_metadata = {
                "Description": "Stack (RGB) preprocessed with Visual Processing Pipeline. Developed using Ryven by Emma Sharratt and Dr Rensue Theart",
                "Author": "Emma Sharratt and Dr Rensue Theart",
                "Date": "Pipeline created in 2023",
                'axes': 'ZCYX' #?????
                # "256": RGB_stack.shape[2], #W
                # "257": RGB_stack.shape[1], #H
                # "slices=": RGB_stack.shape[0],
                # "frames=": 1,
                # "channels=": RGB_stack.shape[3],
            }
tiff.imwrite(new_file_path, img_norm, photometric='rgb', imagej=True, metadata=custom_metadata )
# ValueError: ImageJ hyperstack axes must be in TZCYXS order

### Drop down channel options prototype

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QComboBox, QLabel, QVBoxLayout, QWidget

class ComboBoxExample(QMainWindow):
    def __init__(self):
        super().__init__()

        self.old_choice = None
        self.new_choice = None

        self.initUI()

    def initUI(self):
        self.setWindowTitle('Combo Box Example')
        self.setGeometry(100, 100, 300, 200)

        layout = QVBoxLayout()

        self.combo_box = QComboBox()
        self.combo_box.addItems(['Option 1', 'Option 2', 'Option 3'])
        self.combo_box.currentIndexChanged.connect(self.onIndexChanged)

        self.label_old_choice = QLabel('Old Choice: ')
        self.label_new_choice = QLabel('New Choice: ')

        layout.addWidget(self.combo_box)
        layout.addWidget(self.label_old_choice)
        layout.addWidget(self.label_new_choice)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

    def onIndexChanged(self, index):
        self.old_choice = self.new_choice
        self.new_choice = self.combo_box.currentText()

        self.label_old_choice.setText(f'Old Choice: {self.old_choice}')
        self.label_new_choice.setText(f'New Choice: {self.new_choice}')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ComboBoxExample()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QComboBox, QLabel, QVBoxLayout, QWidget

class combodemo(QWidget):
   def __init__(self, parent=None):
      super(combodemo, self).__init__(parent)
      
      layout = QVBoxLayout()
      self.cb = QComboBox()
      self.cb.addItem("C")
      self.cb.addItem("C++")
      self.cb.addItems(["Java", "C#", "Python"])
      self.cb.currentIndexChanged.connect(self.selectionchange)
      
      layout.addWidget(self.cb)
      self.setLayout(layout)
      self.setWindowTitle("combo box demo")

   def selectionchange(self, i):
      print("Items in the list are :")
      
      for count in range(self.cb.count()):
         print(self.cb.itemText(count))
      print("Current index", i, "selection changed ", self.cb.currentText())

def main():
   app = QApplication(sys.argv)
   ex = combodemo()
   ex.show()
   sys.exit(app.exec_())

if __name__ == '__main__':
   main()


Updates
- Add yellow
- Select > None
- add image and change colour

In [ ]:
import sys
import tifffile as tiff
import numpy as np
from qtpy.QtGui import QFont, QImage, QPixmap, QColor
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QLabel, QCheckBox, QPushButton, QSizePolicy, QAbstractItemView 
from qtpy.QtCore import Qt, Signal, QEvent, QTimer, QObject

class ColorChannelWidget(QWidget):
    def __init__(self, img2d):
        super().__init__()
        self.initUI()

        self.img2D = img2d

    def initUI(self):

        # self.old_choice_dict = {
        #      "old_chan_0": None,
        #      "old_chan_1": None,
        #      "old_chan_2": None,
        #      "old_chan_3": None,
        #      "old_chan_4": None,
        #      "old_chan_5": None,
        # }   
        self.old_choice_array = [None, None, None, None, None, None]
        self.color_name = [None, None, None, None, None, None] 
        # self.old_channel_index = None
        self.dropdowns = []
        self.checkboxes = []
        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "cyan": 100,
                "yellow": 100,
                "magenta": 100                
            }
        }
        self.temp_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "cyan": 100,
                "yellow": 100,
                "magenta": 100                
            }
        }

        self.color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        #CHANGED
        'cyan': 4,
        'yellow': 5,
        'magenta': 6
        }

        self.layout1 = QVBoxLayout()

        longest_word_width = max(len(word) for word in self.color_codes.keys())
        col1_layout = QVBoxLayout()
        col2_layout = QVBoxLayout()

        # Add dropdowns to two columns
        for i, color in enumerate(self.color_codes.keys()):
            label = QLabel(f"Channel {i}:")
            label.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            label.setMinimumWidth(longest_word_width)
            dropdown = QComboBox()
            dropdown.addItem("Select")
            dropdown.setStyleSheet("QComboBox QAbstractItemView { background-color: #091C7F }")
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            dropdown.currentIndexChanged.connect(lambda index, i=i, diction=self.temp_dict: self.handle_selection(index, i, diction))
            self.dropdowns.append(dropdown)
            
            row_layout = QHBoxLayout()  # Create a QHBoxLayout for each row
            row_layout.addWidget(label)
            row_layout.addWidget(dropdown)
            
            # create two coloumns 
            if i < len(self.color_codes)/2:
                col1_layout.addLayout(row_layout)  # Add the row layout to the column layout
            else:
                col2_layout.addLayout(row_layout)  # Add the row layout to the column layout

        # Add column layouts to the main layout
        col_layout = QHBoxLayout()
        col_layout.addLayout(col1_layout)
        col_layout.addLayout(col2_layout)
        self.layout1.addLayout(col_layout)

        # warning
        self.warning = QLabel('')
        self.warning.setStyleSheet('font-size: 1px;')
        # self.warning.setStyleSheet('background-color: #BC0000; color: white; font-size: 14px;')
        self.layout1.addWidget(self.warning)
                
        #image
        self.image_label = QLabel()

        # Create layout for Clear Choices button and Confirm Colour Selection checkbox
        self.button_layout = QHBoxLayout()

        # Confirm checkbox
        self.checkbox = QCheckBox("Confirm channel selection ")
        self.checkbox.stateChanged.connect(lambda state, diction=self.temp_dict: self.update_dict(state, diction))
        self.button_layout.addWidget(self.checkbox)

        # Clear button
        self.clear_button = QPushButton("Clear Choices")
        self.clear_button.clicked.connect(lambda: self.clear_choices(self.stack_dict, self.temp_dict))
        self.button_layout.addWidget(self.clear_button)

        # Add button layout to the main layout
        self.layout1.addLayout(self.button_layout)

        # Image
        self.layout1.addWidget(self.image_label)   

        self.setLayout(self.layout1)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index, dicttemp):
        if index != 0:
            # update the specific channel
            self.old_choice_array[channel_index] = self.color_name[channel_index]
            print(self.old_choice_array)
            old_colour = self.old_choice_array[channel_index]
            print(old_colour)
            if old_colour is not None:
                dicttemp["colour"][old_colour] = 100
            
            
            # update dictionary 
            self.color_name[channel_index] = self.dropdowns[channel_index].currentText()
            print(f'old colour_name: {self.old_choice_array}')
            # Check if colour already selected
            self.warning.setText(' ')
            self.warning.setStyleSheet('font-size: 1px;')
            for colour in self.old_choice_array:
                if colour is not None:
                    if self.color_name[channel_index] == colour:
                        self.warning.setText("WARNING! Please ensure no colours are duplicated")
                        style = """
                            background-color: #BC0000;
                            border-style: outset;
                            border-width: 2px;
                            border-radius: 10px;
                            border-color: beige;
                            font: 14px;
                            min-width: 10em;
                            padding: 6px;
                        """
                        self.warning.setStyleSheet(style)

            dicttemp["colour"][self.color_name[channel_index]] = channel_index
            print(f"{self.color_name[channel_index]}: Channel {channel_index}")
            print(self.stack_dict)
            # Uncheck the checkbox when a dropdown is changed
            if self.checkbox.isChecked():
                self.checkbox.setChecked(False)
                self.clear_img()

    def update_dict(self, state, dicttemp):
        if state == 2:  # Checkbox checked state
            # add code to check if same colours are chosen?
            for color in dicttemp["colour"]:
                self.stack_dict["colour"][color] = dicttemp["colour"][color]
                # dicttemp["colour"][color] = 100
            print(f'update stack: {dicttemp}')
            # print(self.stack_dict)   
            self.show_image(self.img2D)             
                # WILL NEED TO SEND TO SPECIAL NODES LAYER

    # def handle_selection(self, index, channel_index, dicttemp):
    #     if index != 0:
    #         color_name = self.dropdowns[channel_index].currentText()
    #         dicttemp["colour"][color_name] = channel_index
    #         # Uncheck the checkbox when a dropdown is changed
    #         if self.checkbox.isChecked():
    #             self.checkbox.setChecked(False)
    #             self.clear_img()
    #         # self.checkbox.setChecked(False)

    # def update_dict(self, state, dicttemp):
    #     if state == 2:  # Checkbox checked state
    #         # add code to check if same colours are chosen?
    #         for color in dicttemp["colour"]:
    #             self.stack_dict["colour"][color] = dicttemp["colour"][color]
    #             # dicttemp["colour"][color] = 100
    #         print(self.stack_dict)   
    #         self.show_image(self.img2D)             
    #             # WILL NEED TO SEND TO SPECIAL NODES LAYER

    def clear_choices(self, dict, dicttemp):
        self.checkbox.setChecked(False)
        for dropdown in self.dropdowns:
            dropdown.setCurrentIndex(0)  # Reset dropdown menu to "None"

        for color in dict["colour"]:
            dict["colour"][color] = 100  # Reset color values to 100
            dicttemp["colour"][color] = 100
                   
        self.old_choice_array = [None, None, None, None, None, None]
        self.color_name = [None, None, None, None, None, None] 
        
        self.clear_img()
    
    def assign_channels_RGB(self, img):
        single_chan = img[:, :, 0]
        # Initialize RGB channels with zeros
        red_channel = np.zeros_like(single_chan)
        green_channel = np.zeros_like(single_chan)
        blue_channel = np.zeros_like(single_chan)

        for color, channel_value in self.stack_dict["colour"].items():
            # Check if the channel is part of the image
            if channel_value != 100:
                # Assign channels based on the color
                if color == "red":
                    red_channel += img[:, :, channel_value]
                elif color == "green":
                    green_channel += img[:, :, channel_value]
                elif color == "blue":
                    blue_channel += img[:, :, channel_value]
                elif color == "cyan":
                    cyan_channel = img[:, :, channel_value]
                    green_channel += cyan_channel
                    blue_channel += cyan_channel
                elif color == "magenta":
                    magenta_channel = img[:, :, channel_value]
                    red_channel += magenta_channel
                    blue_channel += magenta_channel
                elif color == "yellow":
                    yellow_channel = img[:, :, channel_value]
                    red_channel += yellow_channel
                    green_channel += yellow_channel

        # Clip values to ensure they remain within the valid range [0, 255]
        red_channel = np.clip(red_channel, 0, 255)
        green_channel = np.clip(green_channel, 0, 255)
        blue_channel = np.clip(blue_channel, 0, 255)

        # Combine channels back into image data
        rgb_image_stack = np.stack([red_channel, green_channel, blue_channel], axis=2)

        return rgb_image_stack

    
    def show_image(self, old_img):
        # self.resize(800,800)
        img = self.assign_channels_RGB(old_img)
        
        try:
            if img.shape[-1] == 1:
                # Grayscale image
                qt_image = QImage(img.data, img.shape[1], img.shape[0], img.shape[1], QImage.Format_Grayscale8)
                # #print("came here for Sliderwidget")
            elif img.shape[-1] == 3:
                h, w, ch = img.shape
                bytes_per_line = ch * w
                qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888) #Format_RGB888
            elif img.shape[-1] == 4:
                h, w, ch = img.shape
                #print(f"ch: {ch}")
                bytes_per_line = ch * 4
                qt_image = QImage(img.data, w, h, QImage.Format_RGBA8888) #Format_RGB888
            if qt_image is not None:
                # Calculate the target size for scaling
                scale_factor = 0.7  # Increase the scaling factor for clarity
                if qt_image.width() < 400:
                    scale_factor = 1
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                # Use scaledToWidth to reduce the size while maintaining aspect ratio
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                # Set the scaled pixmap
                self.image_label.setPixmap(scaled_pixmap)
                
                # Resize the widget to match the pixmap size
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
            
        except Exception as e:
            print("Error:", e)
    
    def clear_img(self):
         # Create a black image of size 1x1
        clr_img = QImage(1, 1, QImage.Format_RGB888)
        clr_img.setPixelColor(0, 0, QColor(Qt.black))

        self.image_label.setPixmap(QPixmap(clr_img))
        #print(self.width(), self.height())
        # self.resize(200,50)
        # self.node.update_shape() #works the best. But doesnt minimize shape immediately
    

class ReadImage():
    def __init__(self, path):
        super().__init__()
        #Import image
        
        self.image_data = tiff.imread(path)

        # Normalize - generate dimension list (T,Z,H,W,C)
        self.dim = self.id_tiff_dim(path)
        # Reshape - STANDARDIZED 
        self.image_data = ((self.image_data / np.max(self.image_data)) * 255).astype(np.uint8)

        # IF shape: ZCXY ---------------------------------------------------------------
        # Change to TZXYC
        if self.image_data.shape[-3] <= 4:
            
            # dim = [1, 15, 1024, 1024, 3]
            print("Strange dimension")
            #if no time step data 
            if len(self.image_data.shape) == 4: 
                print(f'length {len(self.image_data.shape)}')
                self.image_data = self.image_data[np.newaxis,:,:,:,:]

            # Create an empty array with the specified dimensions
            image_data_stacked = np.empty(self.dim, dtype=np.uint8)

            num_time_frames = self.dim[0]  # Get the number of time frames
            
            for t in range(num_time_frames):  # Iterate over the time frames
                for i in range(self.dim[1]):  # Iterate over the images in each time frame
                    # Get the red, green, and blue channels for the i-th image in the t-th time frame
                    print(i)
                    chan_0 = self.image_data[t, i, 0, :, :]  # Assuming the first dimension is time frame, then image index
                    chan_1 = self.image_data[t, i, 1, :, :]
                    chan_2 = self.image_data[t, i, 2, :, :]
                    
                    # Stack the channels along the last axis
                    image_data_stacked[t, i, :, :, 0] = chan_0
                    image_data_stacked[t, i, :, :, 1] = chan_1
                    image_data_stacked[t, i, :, :, 2] = chan_2   
                    # if have a an alpha channel
                    if self.image_data.shape[2] >= 4:
                        chan_3 = self.image_data[t, i, 3, :, :] 
                        image_data_stacked[t, i, :, :, 3] = chan_3 

                    # if have a an alpha channel
                    if self.image_data.shape[2] == 5:
                        chan_4 = self.image_data[t, i, 4, :, :] 
                        image_data_stacked[t, i, :, :, 4] = chan_4 

            print(image_data_stacked.shape)   

            self.image_data = image_data_stacked 
            self.slice = self.image_data[0,9,:,:,:]
            print(f'shape: {self.slice.shape}, type: {type(self.slice)}')
        # ---------------------------------------------------------------------------------           
        
    def id_tiff_dim(self,f_path):
        tif_file = tiff.TiffFile(f_path)
        # Check for TIFF metadata tags
        metadata = tif_file.pages[0].tags
        if metadata:
            # print("Metadata Tags:")
            for tag_name, tag in metadata.items():
                print(f"{tag_name}: {tag.value}")

            #set dimension to 0 when a new tiff file is processed
            dimension = [1,1,1,1,1] #dim, slices , time
            
            
            #  T Z Y X C  (F, Z, H, W, C)
            #  0 1 2 3 4
            
            if 256 in metadata: #width
                            # Access the tag value directly
                            dimension[3] = metadata[256].value
            if 257 in metadata: #H
                            # Access the tag value directly
                            dimension[2] = metadata[257].value
            if 277 in metadata: #channels
                            # Access the tag value directly
                            dimension[4] = metadata[277].value
            if 259 in metadata:  # Tag for slices
                            print("meta",metadata[259].value)
                            dimension[1] = metadata[259].value
                        
            if 'ImageDescription' in metadata:
                    # Access 'ImageDescription' tag
                    image_description = metadata['ImageDescription']
            
                    # Split the 'ImageDescription' string into lines
                    description_lines = image_description.value.split('\n')
                    # Parse the lines to extract slices and frames information
                    for line in description_lines:
                        # if 262 in metadata:  # Tag for frames
                        #     dimension[4] = metadata[262].value
                        #     print("dim",dimension[4])
                        if line.startswith("slices="):
                            dimension[1] = int(line.split('=')[1]) #slice
                        if line.startswith("frames="):
                            dimension[0] = int(line.split('=')[1]) #frames
                            # print("frames", int(line.split('=')[1]))
                            # print("dim",dimension[4])
                        if line.startswith("channels="):
                            dimension[4] = int(line.split('=')[1]) #frames
                            # print("dim",dimension[4])
                        
        else:
                print("ImageDescription tag not found in metadata.")
                        
        # print(f'Width: {dimension[3]}')
        # print(f'Height: {dimension[2]}')
        # print(f'Channels: {dimension[4]}')
        # print(f"Slices: {dimension[1]}")
        # print(f"Frames: {dimension[0]}")
        # print(f'Dimension: {dimension[0]}')
        # self.dimension=dimension
        set_widg = [1,1]
        set_widg[0] = dimension[0] #t
        set_widg[1] = dimension[1] #z
        # self.SIGNALS.reset_widget.emit(set_widg)
        # self.SIGNALS.image_shape.emit(dimension)
        # self.stack_dict["time_step"]= 1
        # self.zzval= round(set_widg[1]/2)
        # if (dimension[0])==1 or (dimension[1])==1:
        #     set_widg = [1,1]
        #     self.SIGNALS.reset_widget.emit(set_widg)
        #     self.stack_dict["time_step"]=0
        #     self.zzval=0
        print(f'Image dim: {dimension}')
        return dimension

image_filepath = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"

if __name__ == '__main__':
    app = QApplication(sys.argv)
    
    read_img = ReadImage(image_filepath)
    slicedXYC = read_img.slice
    widget = ColorChannelWidget(slicedXYC)
    #send to function which updates channels which then calls the show image
    # widget.show_image(read_img.slice)
    # print(read_img.image_data)
    widget.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QLabel, QCheckBox, QPushButton, QSizePolicy, QAbstractItemView 

class ColorChannelWidget(QWidget):
    def __init__(self, color_codes):
        super().__init__()
        self.color_codes = color_codes
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.dropdowns = []
        self.checkboxes = []
        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }
        self.temp_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }

        longest_word_width = max(len(word) for word in self.color_codes.keys())
        print(longest_word_width)
        for i in range(5):
            channel_layout = QHBoxLayout()
            label = QLabel(f"Channel {i}:")
            label.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            label.setMinimumWidth(longest_word_width)  # Adjust the multiplier to your preference
            dropdown = QComboBox()
            dropdown.addItem("Select")
            dropdown.setStyleSheet("QComboBox QAbstractItemView { background-color: #091C7F }")
            # dropdown.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
            # dropdown.setMinimumWidth(longest_word_width) 
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            # dropdown.setStyleSheet("QComboBox { background-color: white; }")  # Set background color
            dropdown.currentIndexChanged.connect(lambda index, i=i, diction=self.temp_dict: self.handle_selection(index, i, diction))
            channel_layout.addWidget(label)
            channel_layout.addWidget(dropdown)
            layout.addLayout(channel_layout)
            self.dropdowns.append(dropdown)

        # Create layout for Clear Choices button and Confirm Colour Selection checkbox
        button_layout = QHBoxLayout()

        # Confirm checkbox
        self.checkbox = QCheckBox("Confirm channel selection ")
        self.checkbox.stateChanged.connect(lambda state, diction=self.temp_dict: self.update_dict(state, diction))
        button_layout.addWidget(self.checkbox)

        # Clear button
        self.clear_button = QPushButton("Clear Choices")
        self.clear_button.clicked.connect(lambda: self.clear_choices(self.stack_dict, self.temp_dict))
        button_layout.addWidget(self.clear_button)

        # Add button layout to the main layout
        layout.addLayout(button_layout)

        self.setLayout(layout)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index, dicttemp):
        if index != 0:
            color_name = self.dropdowns[channel_index].currentText()
            dicttemp["colour"][color_name] = channel_index
            # Uncheck the checkbox when a dropdown is changed
            self.checkbox.setChecked(False)

    def update_dict(self, state, dicttemp):
        if state == 2:  # Checkbox checked state
            for color in dicttemp["colour"]:
                self.stack_dict["colour"][color] = dicttemp["colour"][color]
                dicttemp["colour"][color] = 100
            print(self.stack_dict)                
                # WILL NEED TO SEND TO SPECIAL NODES LAYER

    def clear_choices(self, dict, dicttemp):
        self.checkbox.setChecked(False)
        for dropdown in self.dropdowns:
            dropdown.setCurrentIndex(0)  # Reset dropdown menu to "None"

        for color in dict["colour"]:
            dict["colour"][color] = 100  # Reset color values to 100
            dicttemp["colour"][color] = 100

if __name__ == '__main__':
    app = QApplication(sys.argv)
    color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        'magenta': 4,
        'cyan': 5
    }
    widget = ColorChannelWidget(color_codes)
    widget.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QLabel, QCheckBox, QPushButton

class ColorChannelWidget(QWidget):
    def __init__(self, color_codes):
        super().__init__()
        self.color_codes = color_codes
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.dropdowns = []
        self.checkboxes = []
        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }
        self.temp_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }

        for i in range(5):
            channel_layout = QHBoxLayout()
            label = QLabel(f"Channel {i}:")
            dropdown = QComboBox()
            dropdown.addItem("None")
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            dropdown.currentIndexChanged.connect(lambda index, i=i, diction=self.temp_dict: self.handle_selection(index, i, diction))
            channel_layout.addWidget(label)
            channel_layout.addWidget(dropdown)
            layout.addLayout(channel_layout)
            self.dropdowns.append(dropdown)

        # Create layout for Clear Choices button and Confirm Colour Selection checkbox
        button_layout = QHBoxLayout()
        
        # Confirm checkbox
        self.checkbox = QCheckBox("Confirm colour channel selection")
        self.checkbox.stateChanged.connect(lambda state,  diction=self.temp_dict: self.update_dict(state, diction))
        button_layout.addWidget(self.checkbox)

        # Clear button
        self.clear_button = QPushButton("Clear Choices")
        self.clear_button.clicked.connect(lambda: self.clear_choices(self.stack_dict, self.temp_dict))
        button_layout.addWidget(self.clear_button)

        # Add button layout to the main layout
        layout.addLayout(button_layout)

        self.setLayout(layout)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index, dicttemp):
        if index != 0:
            color_name = self.dropdowns[channel_index].currentText()
            dicttemp["colour"][color_name] = channel_index
            # Uncheck the checkbox when a dropdown is changed
            self.checkbox.setChecked(False)

    def update_dict(self, state, dicttemp):
        if state == 2:  # Checkbox checked state
            for color in dicttemp["colour"]:
                self.stack_dict["colour"][color] = dicttemp["colour"][color]
                dicttemp["colour"][color] = 100
                # WILL NEED TO SEND TO SPECIAL NODES LAYER

    def clear_choices(self, dict, dicttemp):
        self.checkbox.setChecked(False)
        for dropdown in self.dropdowns:
            dropdown.setCurrentIndex(0)  # Reset dropdown menu to "None"

        for color in dict["colour"]:
            dict["colour"][color] = 100  # Reset color values to 100
            dicttemp["colour"][color] = 100

if __name__ == '__main__':
    app = QApplication(sys.argv)
    color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        'magenta': 4,
        'cyan': 5
    }
    widget = ColorChannelWidget(color_codes)
    widget.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QLabel, QCheckBox, QPushButton

class ColorChannelWidget(QWidget):
    def __init__(self, color_codes):
        super().__init__()
        self.color_codes = color_codes
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.dropdowns = []
        self.checkboxes = []
        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }
        self.temp_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }

        for i in range(5):
            channel_layout = QHBoxLayout()
            label = QLabel(f"Channel {i}:")
            dropdown = QComboBox()
            dropdown.addItem("None")
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            dropdown.currentIndexChanged.connect(lambda index, i=i, diction=self.temp_dict: self.handle_selection(index, i, diction))
            channel_layout.addWidget(label)
            channel_layout.addWidget(dropdown)
            layout.addLayout(channel_layout)
            self.dropdowns.append(dropdown)

        # Confirm checkbox
        self.checkbox = QCheckBox("Confirm colour channel selection")
        self.checkbox.stateChanged.connect(lambda state,  diction=self.temp_dict: self.update_dict(state, diction))
        layout.addWidget(self.checkbox)

        # Clear button
        self.clear_button = QPushButton("Clear Choices")
        self.clear_button.clicked.connect(lambda: self.clear_choices(self.stack_dict, self.temp_dict))
        layout.addWidget(self.clear_button)

        self.setLayout(layout)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index, dicttemp):
        if index != 0:
            color_name = self.dropdowns[channel_index].currentText()
            dicttemp["colour"][color_name] = channel_index
            # Uncheck the checkbox when a dropdown is changed
            self.checkbox.setChecked(False)

    def update_dict(self, state, dicttemp):
        if state == 2:  # Checkbox checked state
            for color in dicttemp["colour"]:
                self.stack_dict["colour"][color] = dicttemp["colour"][color]
                dicttemp["colour"][color] = 100
                # WILL NEED TO SEND TO SPECIAL NODES LAYER

    def clear_choices(self, dict, dicttemp):
        self.checkbox.setChecked(False)
        for dropdown in self.dropdowns:
            dropdown.setCurrentIndex(0)  # Reset dropdown menu to "None"

        for color in dict["colour"]:
            dict["colour"][color] = 100  # Reset color values to 100
            dicttemp["colour"][color] = 100

if __name__ == '__main__':
    app = QApplication(sys.argv)
    color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        'magenta': 4,
        'cyan': 5
    }
    widget = ColorChannelWidget(color_codes)
    widget.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QComboBox, QLabel, QCheckBox, QPushButton

class ColorChannelWidget(QWidget):
    def __init__(self, color_codes):
        super().__init__()
        self.color_codes = color_codes
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.dropdowns = []
        self.checkboxes = []
        self.stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }
        self.temp_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 100,
                "magenta": 100,
                "cyan": 100
            }
        }
       

        for i in range(5):
            label = QLabel(f"Channel {i}:")
            dropdown = QComboBox()
            dropdown.addItem("None")
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            dropdown.currentIndexChanged.connect(lambda index, i=i, diction = self.temp_dict: self.handle_selection(index, i, diction))
            layout.addWidget(label)
            layout.addWidget(dropdown)
            self.dropdowns.append(dropdown)

        # Confrim checkbox
        self.checkbox = QCheckBox("Confirm colour channel selection")
        self.checkbox.stateChanged.connect(lambda state,  diction = self.temp_dict : self.update_dict(state, diction))
        layout.addWidget(self.checkbox)


        self.clear_button = QPushButton("Clear Choices")
        self.clear_button.clicked.connect(lambda: self.clear_choices(self.stack_dict, self.temp_dict))
        layout.addWidget(self.clear_button)

        print(self.stack_dict)
        self.setLayout(layout)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index, dicttemp):
        if index != 0:
            color_name =self.dropdowns[channel_index].currentText()
            dicttemp["colour"][color_name] = channel_index
            print(f"{color_name}: Channel {channel_index}")
            print(self.stack_dict)
            # Uncheck the checkbox when a dropdown is changed
            self.checkbox.setChecked(False)

    def update_dict(self, state, dicttemp):
        if state == 2:  # Checkbox checked state
            for color in dicttemp["colour"]:
                self.stack_dict["colour"][color] = dicttemp["colour"][color]
                dicttemp["colour"][color] = 100
                #WILL NEED TO SEND TO SPECIAL NODES LAYER
            
            print("Dictionary updated:", self.stack_dict)
            print("Dictionary temporary cleared:", dicttemp)

        
    def clear_choices(self, dict, dicttemp):
        self.checkbox.setChecked(False)
        for dropdown in self.dropdowns:
            dropdown.setCurrentIndex(0)  # Reset dropdown menu to "None"
        
        for color in dict["colour"]:
            dict["colour"][color] = 100  # Reset color values to 100
            dicttemp["colour"][color] = 100
        
        print("Choices cleared and color values reset to 100:", self.stack_dict)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        'magenta': 4,
        'cyan': 5
    }
    widget = ColorChannelWidget(color_codes)
    widget.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QComboBox, QLabel

class ColorChannelWidget(QWidget):
    def __init__(self, color_codes):
        super().__init__()
        self.color_codes = color_codes
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.dropdowns = []
        self.stack_dict = {'time_step':0, 'red':100, 'green':100, 'blue':100, 'magenta':100, 'cyan':100 }

        for i in range(5):
            label = QLabel(f"Channel {i}:")
            dropdown = QComboBox()
            dropdown.addItem("None")
            for color_code in self.color_codes.keys():
                dropdown.addItem(color_code)
            dropdown.currentIndexChanged.connect(lambda index, i=i: self.handle_selection(index, i))
            layout.addWidget(label)
            layout.addWidget(dropdown)
            self.dropdowns.append(dropdown)

        print(self.stack_dict)
        self.setLayout(layout)
        self.setWindowTitle('Color Channel Selector')

    def handle_selection(self, index, channel_index):
        if index != 0:
            color_name =self.dropdowns[channel_index].currentText()
            self.stack_dict[color_name] = channel_index
            print(f"{color_name}: Channel {channel_index}")
            print(self.stack_dict)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    color_codes = {
        'red': 1,
        'green': 2,
        'blue': 3,
        'magenta': 4,
        'cyan': 5
    }
    widget = ColorChannelWidget(color_codes)
    widget.show()
    sys.exit(app.exec_())


Con2 experimenting different channels.

Find shape 

Con2

In [ ]:
import tifffile as tiff
from PyQt5.QtGui import QPixmap, QImage
import numpy as np

path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"
tif_file = tiff.TiffFile(path)
image = tiff.imread(path)

# metadata = tif_file.pages[0].tags
# if metadata:
#      # print("Metadata Tags:")
#     for tag_name, tag in metadata.items():
#         print(f"{tag_name}: {tag.value}")

print(image.shape)

# slice
img = image[5, :, :, :]

# original code
# h, w, ch = img.shape
# manipulate:
# how to deal with this / generalize?



Tried with Numpy but didnt work

In [ ]:
import tifffile as tiff
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QPixmap, QImage

path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"
tif_file = tiff.TiffFile(path)
image1 = tiff.imread(path)
image = ((image1 / np.max(image1)) * 255).astype(np.uint8)

print(image.shape)
dim = [1, 15, 1024, 1024, 3]
if image.shape[-3] <= 4:
    print("Strange dimension")
    # If no time step data
    if len(image.shape) == 4: 
        image = image[np.newaxis, :, :, :, :]

    # Create an empty array with the specified dimensions
    image_data_stacked = np.empty(dim, dtype=np.uint8)

    num_time_frames = dim[0]  # Get the number of time frames
    
    # Reshape the image array to simplify stacking
    reshaped_image = image.reshape(dim[0], dim[1], dim[2], dim[3], -1)

    # Stack the channels along the last axis
    image_data_stacked[:, :, :, :, :] = reshaped_image[:, :, :, :, :3]

    print(image_data_stacked.shape)

class Widget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            h, w, ch  = img.shape
            bytes_per_line = ch * w 
            qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Create an instance of the QApplication object
app = QApplication(sys.argv)

# Create an instance of the widget
widget = Widget()

# Show the image in the widget
widget.show_image(image_data_stacked[0, 9])

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())


Getting ZCXY To ZXYC

In [ ]:
import tifffile as tiff
import numpy as np

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QPixmap, QImage
import numpy as np
import matplotlib.pyplot as plt

path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"
tif_file = tiff.TiffFile(path)
image1 = tiff.imread(path)
image = ((image1 / np.max(image1)) * 255).astype(np.uint8)
print(image.shape)
dim = [1, 15, 1024, 1024, 3]
if image.shape[-3] <= 4:
    print("Strange dimension")
    #if no time step data 
    if len(image.shape) ==4: 
        image = image[np.newaxis,:,:,:,:]

    # Create an empty array with the specified dimensions
    image_data_stacked = np.empty(dim, dtype=np.uint8)

    num_time_frames = dim[0]  # Get the number of time frames
    
    for t in range(num_time_frames):  # Iterate over the time frames
        for i in range(dim[1]):  # Iterate over the images in each time frame
            # Get the red, green, and blue channels for the i-th image in the t-th time frame
            print(i)
            red_channel = image[t, i, 0, :, :]  # Assuming the first dimension is time frame, then image index
            green_channel = image[t, i, 1, :, :]
            blue_channel = image[t, i, 2, :, :]
            
            # Stack the channels along the last axis
            image_data_stacked[t, i, :, :, 0] = red_channel
            image_data_stacked[t, i, :, :, 1] = green_channel
            image_data_stacked[t, i, :, :, 2] = blue_channel

    print(image_data_stacked.shape)
    
#     print("strange dimension")

#     image_data_stacked = np.empty(dim[1:], dtype=np.uint8)
#     print(dim[1:])
#     # for y in range(dim[0]):
#     for i in range(dim[1]):
#             # Get the red, green, and blue channels for the i-th image
#             red_channel = image[i, 0, :, :]
#             green_channel = image[i, 1, :, :]
#             blue_channel = image[i, 2, :, :]
            
#             # Stack the channels along the last axis
#             image_data_stacked[i] = np.stack([red_channel, green_channel, blue_channel], axis=2)
    

# print(image_data_stacked.shape)

class Widget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            h, w, ch  = img.shape
            bytes_per_line = ch * w 
            # qt_image = QImage(img.data, w, h, QImage.Format_RGB888)Format_RGB16
            qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Create an instance of the QApplication object
app = QApplication(sys.argv)

# Create an instance of the widget
widget = Widget()

# Show the image in the widget
widget.show_image(image_data_stacked[0,9,:,:,:])

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())



Handle magenta

Use numpy to optimize computation

Just for the slice. Leave the individual channels

All code:

In [ ]:
def assign_channels_RGB(self, img):
        single_chan = img[:,:,0]
        # Initialize RGB channels with zeros
        red_channel = np.zeros_like(single_chan)
        green_channel = np.zeros_like(single_chan)
        blue_channel = np.zeros_like(single_chan)

        
        for color, channel_value in self.stack_dict["colour"].items():
                #RGB
                if self.stack_dict["colour"]["red"] != 100: 
                        red_channel = img[:,:,channel_value] 
                if self.stack_dict["colour"]["green"] != 100: 
                        green_channel = img[:,:,channel_value] 
                if self.stack_dict["colour"]["blue"] != 100: 
                        blue_channel = img[:,:,channel_value] 
                #CYM
                if self.stack_dict["colour"]["cyan"] != 100: 
                        cyan_channel = img[:,:,channel_value] 
                        green_channel = np.add(red_channel, cyan_channel)
                        blue_channel = np.add(blue_channel, cyan_channel)
                       

                #repeat for yellow and magenta
                        
        # Clip values to ensure they remain within the valid range [0, 255]
        red_channel = np.clip(red_channel, 0, 255)
        green_channel = np.clip(green_channel, 0, 255)
        blue_channel = np.clip(blue_channel, 0, 255)


        # Combine channels back into image data
        rgb_image_stack = np.stack([red_channel, green_channel, blue_channel], axis=2)
                
                        

In [ ]:
import tifffile as tiff
import numpy as np

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QPixmap, QImage
import numpy as np
import matplotlib.pyplot as plt

path = r"/home/emma/ve/vipp/SkripsieProject/skripsie_projects/Control_2.tif"
tif_file = tiff.TiffFile(path)
image = tiff.imread(path)

# metadata = tif_file.pages[0].tags
# if metadata:
#      # print("Metadata Tags:")
#     for tag_name, tag in metadata.items():
#         print(f"{tag_name}: {tag.value}")

print(image.shape)
print(np.max(image[9,0,:,:]))
print(np.max(image[9,1,:,:]))
print(np.max(image[9,2,:,:]))
# Slice
img = image[5, :, :, :]
# Normlize for 8bit PyQt widget
img_norm = ((img / np.max(img)) * 255).astype(np.uint8)

print("img_norm")
print(img_norm.shape)

# Extract individual channels
magenta_channel = img_norm[0,:,:]
green_channel = img_norm[1,:,:]
blue_channel = img_norm[2,:,:]
red_channel = np.zeros_like(blue_channel)

# red_channel = img_norm[0,:,:]
# green_channel = img_norm[1,:,:]
# blue_channel = img_norm[2,:,:]

# Add magenta channel to red and blue channels
red_channel = np.add(red_channel, magenta_channel)
blue_channel = np.add(blue_channel, magenta_channel)

# Clip values to ensure they remain within the valid range [0, 255]
red_channel = np.clip(red_channel, 0, 255)
blue_channel = np.clip(blue_channel, 0, 255)

# Combine channels back into image data
rgb_image_stack = np.stack([red_channel, green_channel, blue_channel], axis=2)
# # rgb_image = np.concatenate((red_channel, green_channel, blue_channel), axis=-1)
# print("stack")
# print(rgb_image_stack.shape)

# # print(np.max(rgb_image_stack[0,:,:]))
# # print(np.max(rgb_image_stack[1,:,:]))
# # print(np.max(rgb_image_stack[2,:,:]))



# rgb_image_tansp = img_norm.reshape(1024,1024,3)
# image_bytes = rgb_image_transp.tobytes()
# rgb_image_tansp = np.transpose(img_norm, (1, 2, 0))
# rgb_image_tansp = np.moveaxis(img_norm, 0, 2)
print(np.max(rgb_image_stack[:,:, 0]))
print(np.max(rgb_image_stack[:,:,1]))
print(np.max(rgb_image_stack[:,:,2]))
# rgb_image_tansp = rgb_image_stack
print("transpose")
print(rgb_image_stack.shape)
print(type(rgb_image_stack))
# plt.imshow(rgb_image)
# plt.axis('off')  # optional: turn off axis
# plt.show()
# print(rgb_image.shape)


def assign_channels_RGB(img):
    stack_dict = {
            "time_step": 0,
            "colour": {
                "red": 100,
                "green": 100,
                "blue": 1,
                "cyan": 100,
                "yellow": 2,
                "magenta": 0
                
            }
        }
    
    single_chan = img[:, :, 0]
    # Initialize RGB channels with zeros
    red_channel = np.zeros_like(single_chan)
    green_channel = np.zeros_like(single_chan)
    blue_channel = np.zeros_like(single_chan)

    for color, channel_value in stack_dict["colour"].items():
        # Check if the channel is part of the image
        if channel_value != 100:
            # Assign channels based on the color
            if color == "red":
                red_channel += img[:, :, channel_value]
            elif color == "green":
                green_channel += img[:, :, channel_value]
            elif color == "blue":
                blue_channel += img[:, :, channel_value]
            elif color == "cyan":
                cyan_channel = img[:, :, channel_value]
                green_channel += cyan_channel
                blue_channel += cyan_channel
            elif color == "magenta":
                magenta_channel = img[:, :, channel_value]
                red_channel += magenta_channel
                blue_channel += magenta_channel
            elif color == "yellow":
                yellow_channel = img[:, :, channel_value]
                red_channel += yellow_channel
                green_channel += yellow_channel

    # Clip values to ensure they remain within the valid range [0, 255]
    red_channel = np.clip(red_channel, 0, 255)
    green_channel = np.clip(green_channel, 0, 255)
    blue_channel = np.clip(blue_channel, 0, 255)

    # Combine channels back into image data
    rgb_image_stack = np.stack([red_channel, green_channel, blue_channel], axis=2)

    return rgb_image_stack

class Widget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            h, w, ch  = img.shape
            bytes_per_line = ch * w 
            # qt_image = QImage(img.data, w, h, QImage.Format_RGB888)Format_RGB16
            qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Create an instance of the QApplication object
app = QApplication(sys.argv)

# Create an instance of the widget
widget = Widget()

# Show the image in the widget
rgb_assigned = assign_channels_RGB(rgb_image_stack)
widget.show_image(rgb_assigned)

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())

In [ ]:
# Extract individual channels
magenta_channel = img[0]
red_channel = img[1]
blue_channel = img[2]

# Add magenta channel to red and blue channels
red_channel = np.add(red_channel, magenta_channel)
blue_channel = np.add(blue_channel, magenta_channel)

# Clip values to ensure they remain within the valid range [0, 255]
red_channel = np.clip(red_channel, 0, 255)
blue_channel = np.clip(blue_channel, 0, 255)

# Combine channels back into image data
rgb_image = np.stack([red_channel, np.zeros_like(red_channel), blue_channel])
print(rgb_image.shape)



# Normlize for 8bit PyQt widget
rgb_image = (rgb_image / np.max(img) * 255).astype(np.uint8)
print(rgb_image.shape)

Whole stack?

In [ ]:
# Extract individual channels
magenta_channel = image[0, :, :, :]
red_channel = image[1, :, :, :]
blue_channel = image[2,  :, :, :]

# Add magenta channel to red and blue channels
red_channel = np.add(red_channel, magenta_channel)
blue_channel = np.add(blue_channel, magenta_channel)

# Clip values to ensure they remain within the valid range [0, 255]
red_channel = np.clip(red_channel, 0, 255)
blue_channel = np.clip(blue_channel, 0, 255)

# Combine channels back into image data
image_data_processed = np.stack([magenta_channel, red_channel, blue_channel])
np.concatenate((self.red_stack, self.gr_stack, self.blue_stack), axis=-1)
print(image.shape)
# slice,
img = image_data_processed[10, :, :, :]

# Normlize for 8bit PyQt widget
img = (img / np.max(img) * 255).astype(np.uint8)
print(img.shape)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QPixmap, QImage
import numpy as np

class YourWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            ch, h, w = img.shape
            bytes_per_line = ch * w
            qt_image = QImage(img.data, w, h, QImage.Format_RGB888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Create an instance of the QApplication object
app = QApplication(sys.argv)

# Read the image
# Assuming you have your image data in 'image_data'
# image_data = np.random.rand(3, 1024, 1024)  # Example random image data

# Create an instance of the widget
widget = YourWidget()

# Show the image in the widget
widget.show_image(rgb_image)

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())


Flybrain

In [ ]:
import tifffile as tiff

path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\flybrain.tif"
tif_file = tiff.TiffFile(path)
image = tiff.imread(path)

metadata = tif_file.pages[0].tags
if metadata:
     # print("Metadata Tags:")
    for tag_name, tag in metadata.items():
        print(f"{tag_name}: {tag.value}")

print(image.shape)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QPixmap, QImage
import tifffile as tiff

class YourWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            qt_image = QImage(img.data, img.shape[1], img.shape[0], img.shape[1] * img.shape[2], QImage.Format_RGB888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Read the image
path = r"C:\Users\dell\OneDrive\Documents\Em\2023\Skripsie\Development\Data\Control_2.tif"
image = tiff.imread(path)

# Create an instance of the widget
widget = YourWidget()

# Show the image in the widget
widget.show_image(image)

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QMessageBox
from PyQt5.QtGui import QPixmap, QImage

class YourWidget(QWidget):
    def __init__(self):
        super().__init__()
        
        # Create a QLabel to display the image
        self.image_label = QLabel(self)
        
        # Create a layout to arrange the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)
    
    def show_image(self, img):
        try:
            if img.shape[-1] == 1:
                qt_image = QImage(img.data, img.shape[1], img.shape[0], img.shape[1], QImage.Format_Grayscale8)
            elif img.shape[-1] == 3:
                h, w, ch = img.shape
                bytes_per_line = ch * w
                qt_image = QImage(img.data, w, h, bytes_per_line, QImage.Format_RGB888)
            elif img.shape[-1] == 4:
                h, w, ch = img.shape
                bytes_per_line = ch * 4
                qt_image = QImage(img.data, w, h, QImage.Format_RGBA8888)
            
            if qt_image is not None:
                scale_factor = 0.7
                if qt_image.width() < 400:
                    scale_factor = 1.2
                if qt_image.width() > 900:
                    scale_factor = 0.5
                target_width = int(qt_image.width() * scale_factor)
                scaled_pixmap = QPixmap.fromImage(qt_image).scaledToWidth(target_width)
                
                self.image_label.setPixmap(scaled_pixmap)
                self.resize(scaled_pixmap.width(), scaled_pixmap.height())
                
        except Exception as e:
            print("Error:", e)

# Create a function to display the pop-up window
def show_popup():
    popup = QMessageBox()
    popup.setWindowTitle("Pop-up Window")
    popup.setText("Your message goes here!")
    popup.exec_()

# Create an instance of the widget
widget = YourWidget()

# Show the pop-up window
show_popup()

# Read the image and display it
image_data = read_tiff_image()  # Replace this with your function to read the TIFF image
widget.show_image(image_data)

# Display the widget
widget.show()

# Run the application
sys.exit(app.exec_())
